# Session 12 Assignment 

## Víctor Vega Sobral
## Santiago Souto Ortega

---

## Implementing Von Neumann Neighborhood for PSO

### 1. Von Neumann Explanation

In standard PSO, each particle is influenced by:

* Its best personal position (cognitive component).
* Best global position of all the swarm (social component).

In a Von Neumann Neighborhood, particles are organized in a 2D net, where each particle is only connected tu its inmediate neighbors (north, south, west and east). Social component is based in best position found inside its neighborhood, not in all the swarm.

With this architecture, more diverse exploration in the search space is made, with more resistance to local optima and slower convergence, but potentially more robust.


## 1. Step by Step Implementation

In [4]:
import numpy as np
import math

class ParticleSwarmOptimizationVN(object):
    """
    Particle Swarm Optimization (PSO) implementation with Von Neumann neighborhood.
    
    This implementation arranges particles in a 2D grid, where each particle is
    influenced by its own best position and the best positions of its immediate
    neighbors (north, south, east, west) in the grid.
    """

    def __init__(
        self,
        objective_function: callable,
        n_particles: int = 25,  # Preferably a perfect square for grid arrangement
        dimensions: int = 2,
        bounds: tuple[float, float] = (0, 5),
        w: float = 0.8,
        c1: float = 0.1,
        c2: float = 0.1,
        seed: int = 100
    ):
        """
        Initialize the PSO algorithm with Von Neumann neighborhood.

        Parameters:
        -----------
        objective_function : function
            The function to be minimized
        n_particles : int
            Number of particles in the swarm (preferably a perfect square)
        dimensions : int
            Number of dimensions in the search space
        bounds : tuple
            (min, max) bounds for each dimension
        w : float
            Inertia weight - controls influence of previous velocity
        c1 : float
            Cognitive parameter - controls attraction to personal best
        c2 : float
            Social parameter - controls attraction to neighborhood best
        seed : int
            Random seed for reproducibility
        """
        self.objective_function = objective_function
        self.n_particles = n_particles
        self.dimensions = dimensions
        self.bounds = bounds
        self.w = w
        self.c1 = c1
        self.c2 = c2

        # Set random seed for reproducibility
        np.random.seed(seed)

        # Initialize particles' positions and velocities
        self.X = np.random.rand(dimensions, n_particles) * \
            (bounds[1] - bounds[0]) + bounds[0]
        self.V = np.random.randn(dimensions, n_particles) * 0.1

        # Initialize personal best positions and objective values
        self.pbest = self.X.copy()
        self.pbest_obj = self.evaluate(self.X)

        # Create grid topology and find neighbors
        self.grid_size = int(np.ceil(np.sqrt(n_particles)))
        self.neighbors = self._create_von_neumann_neighborhood()
        
        # Initialize neighborhood best positions and values
        self.nbest = np.zeros((dimensions, n_particles))
        self.nbest_obj = np.ones(n_particles) * float('inf')
        self._update_nbest()

        # Initialize global best (still tracked, but not used for updates)
        self.gbest = self.pbest[:, self.pbest_obj.argmin()]
        self.gbest_obj = self.pbest_obj.min()

        # Store optimization history
        self.history = {
            'positions': [self.X.copy()],
            'velocities': [self.V.copy()],
            'pbest': [self.pbest.copy()],
            'nbest': [self.nbest.copy()],
            'gbest': [self.gbest.copy()],
            'gbest_obj': [self.gbest_obj]
        }

    def _create_von_neumann_neighborhood(self):
        """
        Create Von Neumann neighborhood topology (north, south, east, west).
        
        Returns:
        --------
        list
            List of neighbor indices for each particle
        """
        neighbors = []
        for i in range(self.n_particles):
            # Convert particle index to 2D grid coordinates
            row = i // self.grid_size
            col = i % self.grid_size
            
            # Find neighbors (north, south, east, west with wrap-around)
            north = ((row - 1) % self.grid_size) * self.grid_size + col
            south = ((row + 1) % self.grid_size) * self.grid_size + col
            east = row * self.grid_size + ((col + 1) % self.grid_size)
            west = row * self.grid_size + ((col - 1) % self.grid_size)
            
            # Ensure indices are within range
            particle_neighbors = [
                north if north < self.n_particles else i,
                south if south < self.n_particles else i,
                east if east < self.n_particles else i,
                west if west < self.n_particles else i
            ]
            neighbors.append(particle_neighbors)
        
        return neighbors

    def _update_nbest(self):
        """
        Update neighborhood best positions based on current personal bests.
        """
        for i in range(self.n_particles):
            # Include the particle itself in the neighborhood
            neighborhood = self.neighbors[i] + [i]
            
            # Find the best particle in the neighborhood
            best_idx = neighborhood[np.argmin([self.pbest_obj[j] for j in neighborhood])]
            
            # Update neighborhood best if better
            if self.pbest_obj[best_idx] < self.nbest_obj[i]:
                self.nbest[:, i] = self.pbest[:, best_idx]
                self.nbest_obj[i] = self.pbest_obj[best_idx]

    def evaluate(self, X):
        """
        Evaluate the objective function for all particles.

        Parameters:
        -----------
        X : ndarray
            Particles' positions

        Returns:
        --------
        ndarray
            Objective function values for all particles
        """
        return self.objective_function(X[0], X[1])

    def update(self):
        """
        Perform one iteration of the PSO algorithm with Von Neumann neighborhood.

        Steps:
        1. Update velocities based on inertia, cognitive, and social components
        2. Update positions based on velocities
        3. Evaluate new positions
        4. Update personal and neighborhood bests

        Returns:
        --------
        dict
            Current state of the swarm
        """
        # Generate random coefficients for stochastic behavior
        r1, r2 = np.random.rand(2)

        # Update velocities using neighborhood best instead of global best
        self.V = (self.w * self.V +                             # Inertia component
                  self.c1 * r1 * (self.pbest - self.X) +        # Cognitive component
                  self.c2 * r2 * (self.nbest - self.X))         # Social component (neighborhood)

        # Update positions by adding velocities
        self.X = self.X + self.V
        
        # Enforce bounds if necessary
        self.X = np.clip(self.X, self.bounds[0], self.bounds[1])

        # Evaluate new positions
        obj = self.evaluate(self.X)

        # Update personal best positions and objective values
        mask = (self.pbest_obj >= obj)
        self.pbest[:, mask] = self.X[:, mask]
        self.pbest_obj = np.minimum(self.pbest_obj, obj)

        # Update neighborhood bests
        self._update_nbest()

        # Update global best (still tracked for comparison)
        min_idx = self.pbest_obj.argmin()
        self.gbest = self.pbest[:, min_idx]
        self.gbest_obj = self.pbest_obj[min_idx]

        # Store current state for history
        self.history['positions'].append(self.X.copy())
        self.history['velocities'].append(self.V.copy())
        self.history['pbest'].append(self.pbest.copy())
        self.history['nbest'].append(self.nbest.copy())
        self.history['gbest'].append(self.gbest.copy())
        self.history['gbest_obj'].append(self.gbest_obj)

        # Return current state for visualization
        return {
            'X': self.X,
            'V': self.V,
            'pbest': self.pbest,
            'nbest': self.nbest,
            'gbest': self.gbest,
            'gbest_obj': self.gbest_obj
        }

    def optimize(self, iterations=50):
        """
        Run the PSO algorithm for a specified number of iterations.

        Parameters:
        -----------
        iterations : int
            Number of iterations to run

        Returns:
        --------
        tuple
            (global best position, global best objective value)
        """
        for _ in range(iterations):
            self.update()

        return self.gbest, self.gbest_obj

    def get_history(self):
        """
        Get the optimization history.

        Returns:
        --------
        dict
            Optimization history containing positions, velocities, 
            personal bests, neighborhood bests, global best, and objective values
        """
        return self.history

---

## 2. Test  it  compared  to  the  previous  implementation 
Explain also the functions that you use and why they are used as test cases.

Functions:
- [Wikipedia Functions](https://en.wikipedia.org/wiki/Test_functions_for_optimization)

In [5]:
### Code for testing Von Neumann

---

## 3. Ant Conoly Optimization Pseudocode


### 3.1 ACO Parameters

- G = (V, E): Graph with vertices V and edges E
- τ(i,j): Pheromone level on edge (i,j)
- η(i,j): Heuristic information (1/distance) for edge (i,j)
- α: Pheromone influence parameter
- β: Heuristic influence parameter
- ρ: Pheromone evaporation rate
- m: Number of ants
- Q: Constant for pheromone updates
- source, destination: Start and end nodes of the path
- max_iter: Maximum number of iterations

### 3.2 Initialization

1. For each edge (i,j) in E:
   - τ(i,j) = τ₀ (small positive constant, e.g., 0.1)
2. best_path = NULL
3. best_length = ∞

### 3.3 ACO Main Loop

**For** `iter = 1` **to** `max_iter`:

#### Ant Tour Construction

**For** `k = 1` **to** `m` (for each ant):
- Place ant *k* at the source node  
- `path_k = [source]`  
- `current_node = source`

**While** `current_node ≠ destination`:
- Calculate the selection probability for each unvisited node *j*:

  \[
  p(j) = \frac{[\tau(current\_node, j)]^\alpha \cdot [\eta(current\_node, j)]^\beta}{\sum [\tau(current\_node, l)]^\alpha \cdot [\eta(current\_node, l)]^\beta}
  \]

- Select the next node based on these probabilities (roulette wheel method)  
- Add the selected node to `path_k`  
- `current_node = selected node`

- Calculate `length_k` (total length of the found path)  
- **If** `length_k < best_length`:
  - `best_path = path_k`
  - `best_length = length_k`


### 3.4 Pheromone update

#### Evaporation
For each edge (i,j):
  - τ(i,j) = (1-ρ) * τ(i,j)
   
#### Deposit
For each ant k:
  For each edge (i,j) in path_k:
    - Δτ(i,j) = Q / length_k
    - τ(i,j) = τ(i,j) + Δτ(i,j)

### 3.5 Return
Return best_path, best_length

### 3.6 Helper Functions

SelectNextNode(current_node, unvisited_nodes, τ, η, α, β):
1. Calculate total sum:
   - sum = Σ [τ(current_node,j)]^α * [η(current_node,j)]^β for all j in unvisited_nodes
2. For each node j in unvisited_nodes:
   - Calculate probability p(j) = [τ(current_node,j)]^α * [η(current_node,j)]^β / sum
3. Generate random number r between 0 and 1
4. Select node using roulette wheel method based on p(j) and r
5. Return selected node

CalculatePathLength(path, G):
1. length = 0
2. For i = 0 to length(path)-2:
   - length += distance between path[i] and path[i+1] in G
3. Return length

---

## 4. Solving the TSP With Pseudocode